<br/><font size=10>Data organization</font><br/>

# Introduction

In this Notebook, we use the data from EEG Motor Movement/Imagery Dataset (EEG-MMIDB)[<sup>1</sup>](#refer-anchor-1). We offer the cleaned dataset that extract from the EEG-MMIDB that contains 109 subjects. In this Notebook, for simplicity, we take a subject as a subset to show how the code works.


<!-- > Dataset_1: the data of the first subject in the EEG-MMIDB.  
> Dataset_2: data of all 109 subjects of the EEG-MMIDB.

* Note: in the following models of this BCI Notebook, we will use Dataset_1 to run examples for the computational advantage. -->

## Dataset description

The original dataset description can be find here: https://archive.physionet.org/pn4/eegmmidb/

After our clearning and sorting, each .npy file comprised of the data of one subject. The data shape of each npy file is [N, 65], the first 64 columns represent the readouts of 64 EEG channel, the last column denotes the class/intent label. The row denotes time-points in signal collection and one row represents one readout at a specific time-point. In this Notebook, we call each row a instance. The sampling rate in EEG-MMIDB is 160 Hz, which means that the equipment can generate 160 instances/rows/time-points in each second.


The N varis for different subjects, but N should be 259,520 or 255,680. This is the inherent difference in the original dataset. Recall that the sampling rate is 160 Hz, thus, some trials last for 4.1 seconds while others last for 4.2 seconds: 4.1 seconds (656=4.1 $\times$ 160 instances) or 4.2 seconds (672 = 4.2 $\times$  160 instancs). It is suggested to segment the signals in each second. 


Based on the experimental setting, we split all EEG signals into 11 different cognitive intentions as follows. In which, the intentions with *image* represents the subject only image the action but not move in reality: these four intentions (labelled by 4, 5, 8, and 9) are strictly *movement imagery EEG*. The residual intentions are rather the mental states that the user was conducting a specific action.
> Labels:  
0: open eyes,  
1: close eyes.  
2: left hand,   
3: right hand.  
4: image left hand,   
5: image right hand.  
6: open fists,   
7:open feet.  
8: image fist,   
9: image feet.  
10: rest.

## Load dataset

Next, we show how to load the data in 2 commands!

In [2]:
import numpy as np
dataset_1=np.load('1.npy')
print('The shape of Dataset_1:', dataset_1.shape)
dataset_1

The shape of Dataset_1: (259520, 65)


array([[-16, -29,   2, ..., -11,  15,   0],
       [-56, -54, -27, ...,   1,  21,   0],
       [-55, -55, -29, ...,  18,  35,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9]], dtype=int64)

As we can see from above, the data of subject 1 consists of 259520 instances and 64 channels while the last column denotes the class label (ground truth).

We first introduce several terms about the data orgnization:
* Instance (time step or time point). Each instance indicates a list of values which are collected at a single time point or sampling point. For example, there will be 160 instances in 1 second for the equipment with 160 Hz sampling rate. We use instance and time point interchangeably.

* Segment (sample). Each segment contains multiple continue instances which can represent a specific event/state of brain signals. The length of a segment is called time window. We use segment and sample interchangeably.


# Resampling

Next, we introduce the implementation of resampling in different conditions.

*In this Notebook,we do not need resampling since our data has appropriate sampling frequency and is already balanced.*

# Filtering

Sometimes we need to filter the EEG signals into frequency bands of interests, in order to find the most informative and distinguishable patterns. 

The EEG signals collected from any typical EEG hardware have several nonoverlapping frequency bands (Delta, Theta, Alpha, Beta, and Gamma) based on the strong intraband correlation with a distinct behavioral state. Each EEG pattern contains signals associated with particular brain information. The degree of awareness denotes the perception of individuals when presented with external stimuli. It is mainly defined in physiology instead of psychology.[<sup>3</sup>](#refer-anchor-3)  

Each frequency band represents a brain state and a qualitative assessment of awareness:
* Delta pattern (0.5--4 Hz) corresponds to deep sleep when the subject has lower awareness.  
- Theta pattern (4--8 Hz) corresponds to light sleep in the realm of low awareness.   
* Alpha pattern (8--12 Hz) mainly occurs during eyes closed and deeply relaxed state and corresponds to the medium awareness. 
* Beta pattern (12--30 Hz) is the dominant rhythm while the eyes of the subject are open and is associated with high awareness. Beta patterns capture most of our daily activities (such as eating, walking, and talking). 
* Gamma pattern (30--100 Hz) represents the co-interaction of several brain areas to carry out a specific motor and cognitive function. 

Next, we provide the filtering codes taking the Delta band as an example (we don't need filtering in this Notebook, just provide the code in case the readers need it). The filter used here is 3-order band-pass butterworth filter. Please adjust sampling frequency (fs), lowcut and highcut to customize your own filter. 

In [3]:
import myimporter  # active import from inside notebook
from BCI_functions import *  # BCI_functions.ipynb contains some functions we might use multiple times in this Notebook 

n_fea = 64  # 64 channels
label = dataset_1[:, n_fea: n_fea+1]  # seperate label from feature
feature = dataset_1[:, 0:n_fea] 
feature_f=[]  # feature after filtering

# EEG Delta pattern decomposition
for i in range(feature.shape[1]):
    x = feature[:, i]
    fs = 160.0
    lowcut = 0.5
    highcut = 4.0
    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=3)
    feature_f.append(y) 
    
feature_f=np.array(feature_f).T
print('The shape of filtered feature:',feature_f.shape)

data_f=np.hstack((feature_f,label))  # stack label to filtered feature 
print("The shape of dataset_1 after filtering:",data_f.shape)

importing Jupyter notebook from BCI_functions.ipynb
The shape of filtered feature: (259520, 64)
The shape of dataset_1 after filtering: (259520, 65)


# Reference

<div id="refer-anchor-1"></div>

- [1]  [Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220](http://circ.ahajournals.org/cgi/content/full/101/23/e215)


<div id="refer-anchor-2"></div>

- [3]  [Zhang X, Yao L, Wang X, et al. A survey on deep learning-based non-invasive brain signals: recent advances and new frontiers[J]. Journal of Neural Engineering, 2020.](https://arxiv.org/abs/1905.04149)

